In [1]:
import pandas as pd
import glob
from padelpy import padeldescriptor

In [2]:
def make_ml_df(df3):

    df3_selection = df3[['canonical_smiles', 'molecule_chembl_id']]
    df3_selection.to_csv('python_notebooks/data/molecule.smi', sep='\t', index=False, header=False)

    xml_files = glob.glob("python_notebooks/data/xml_files/*.xml")
    xml_files.sort()

    FP_list = ['AtomPairs2DCount',
                'AtomPairs2D',
                'EState',
                'CDKextended',
                'CDK',
                'CDKgraphonly',
                'KlekotaRothCount',
                'KlekotaRoth',
                'MACCS',
                'PubChem',
                'SubstructureCount',
                'Substructure']

    fp = dict(zip(FP_list, xml_files))

    fingerprint = 'PubChem'

    fingerprint_output_file = ''.join(['python_notebooks/data/PubChem_fingerprints/',fingerprint,'.csv']) #Substructure.csv
    fingerprint_descriptortypes = fp[fingerprint]

    padeldescriptor(mol_dir='python_notebooks/data/molecule.smi', 
                    d_file=fingerprint_output_file, 
                    descriptortypes= fingerprint_descriptortypes,
                    detectaromaticity=True,
                    standardizenitro=True,
                    standardizetautomers=True,
                    threads=2,
                    removesalt=True,
                    log=True,
                    fingerprints=True)

    fingerprint = 'PubChem'
    fingerprint_output_file = ''.join(['python_notebooks/data/PubChem_fingerprints/',fingerprint,'.csv']) 

    descriptors = pd.read_csv(fingerprint_output_file)

    df_ml = pd.concat([descriptors, df3['pIC50']], axis=1).dropna()

    # Y = df_ml['pIC50']
    # X = df_ml.drop(['Name','pIC50'], axis=1)
    # selection = VarianceThreshold(threshold=(.8 * (1 - .8)))    
    # X = selection.fit_transform(X)
    # X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
    # model = RandomForestRegressor(n_estimators=100)
    # model.fit(X_train, Y_train)
    # r2 = model.score(X_test, Y_test)

    return(df_ml)

In [3]:
df3 = pd.read_csv('python_notebooks/data/cleaned_bioactivity_data.csv')

In [4]:
make_ml_df(df3)

,Name,PubchemFP0,PubchemFP1,PubchemFP2,PubchemFP3,PubchemFP4,PubchemFP5,PubchemFP6,PubchemFP7,PubchemFP8,...,PubchemFP872,PubchemFP873,PubchemFP874,PubchemFP875,PubchemFP876,PubchemFP877,PubchemFP878,PubchemFP879,PubchemFP880,pIC50
0,CHEMBL2094865,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4.527244
1,CHEMBL2098433,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4.643974
2,CHEMBL2098408,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4.279014
3,CHEMBL164087,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4.290730
4,CHEMBL3774692,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7.920819
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
553,CHEMBL4790440,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,6.266883
554,CHEMBL4782583,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7.612610
555,CHEMBL4759858,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7.030584
556,CHEMBL4798603,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5.000000
